In [3]:
from mne.io import read_raw_edf

from try_fft3 import current_signal

file_path = "01IS-1-ADP.edf"
raw = read_raw_edf(file_path, preload=True)

sampling_rate = int(raw.info['sfreq'])
signal = raw.get_data(picks="eeg")[0]

Extracting EDF parameters from /media/artem/data/Dev/Sleep_analysis/01IS-1-ADP.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7529215  =      0.000 ... 29410.996 secs...


In [4]:
from signal_periodizer.periodizer import SignalPeriodizer

periodizer = SignalPeriodizer(point_per_period = sampling_rate * 20 )
last_period = periodizer.last_period_for(signal=signal)

print(f"{last_period=}")

last_period=1469


In [9]:
from tqdm import tqdm
from fft_analyser.homebrew import FftHomebrew
from fft_analyser.welch import FftWelch

fft_analyser = FftWelch(sampling_rate=sampling_rate)

all_fft_analysis = {}
all_signal_period = {}

for period in tqdm(range(last_period + 1)):
    current_signal = periodizer.period_for(signal=signal, n_of_period=period)
    all_signal_period[period] = current_signal

    result = fft_analyser.compute_for(signal=current_signal)
    all_fft_analysis[period] = result


100%|██████████| 1470/1470 [00:00<00:00, 3193.19it/s]


In [11]:
from matplotlib import gridspec, pyplot as plt


def compute_graph_for_a_period(fft_analysis, signal_period, period=0):
    fft = fft_analysis[period]
    signal = signal_period[period]

    fig = plt.figure(figsize=(10, 8))
    gs = gridspec.GridSpec(2, 1, height_ratios=[1, 4])
    ax0 = fig.add_subplot(gs[0])
    ax1 = fig.add_subplot(gs[1])

    ax0.plot(signal)
    ax1.plot(fft.keys(), fft.values())

    plt.show()

#
# compute_graph_for_a_period(fft_analysis=all_fft_analysis,
#                            signal_period=all_signal_period,
#                            period=0)


In [12]:
import ipywidgets as widgets
from ipywidgets import interact

def interactive_plot():
    period_slider = widgets.IntSlider(value=52, min=0, max=last_period, step=1, description="Period:")
    interact(compute_graph_for_a_period,
             fft_analysis=widgets.fixed(all_fft_analysis),
             signal_period=widgets.fixed(all_signal_period),
             period=period_slider)


interactive_plot()

interactive(children=(IntSlider(value=52, description='Period:', max=1469), Output()), _dom_classes=('widget-i…